In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/informal-text-dataset/tachygraphy_dataset_v2.xlsx


In [26]:
import emoji
from bs4 import BeautifulSoup
import os
import re
import string
import json

'''For emoji cleaning'''

'For emoji cleaning'

In [27]:
dataset = pd.read_excel('/kaggle/input/informal-text-dataset/tachygraphy_dataset_v2.xlsx')

In [28]:
df=dataset

In [29]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [30]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

In [31]:
def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_contractions(text, contraction_mapping)
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
#     text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [32]:
df['Expanded Meaning']

0        Oh my god, Joint Entrance Examination preparat...
1        Are you up for a break before Union Public Ser...
2        Talk to you later, finishing the Common Admiss...
3        No, that Graduate Aptitude Test in Engineering...
4        What's up? Are you done with your Indian Insti...
                               ...                        
10275    Brother, power cuts in Uttar Pradesh are ongoi...
10276    What the fuck, new tax laws are increasing, ca...
10277    Yo, Adani’s stock is dropping, with more probl...
10278    Brother, new technology IPOs are performing we...
10279    What the fuck, Delhi’s air quality index has h...
Name: Expanded Meaning, Length: 10280, dtype: object

In [33]:
df.isnull().sum()

Informal Text       2
Expanded Meaning    2
dtype: int64

In [34]:
df=df.fillna('')

In [35]:
df['Expanded Meaning'] = df['Expanded Meaning'].astype(str)
df['Expanded Meaning'] = df['Expanded Meaning'].apply(lambda x: text_preprocessing_pipeline(x))

In [36]:
df.tail()

,Informal Text,Expanded Meaning
10275,"bruh, UP’s power cuts, no relief in sight 🔌",brother power cuts in uttar pradesh are ongoin...
10276,"wtf, new tax laws hitting hard, prices gonna j...",what the fuck new tax laws are increasing caus...
10277,"yo, Adani’s stock falling, more issues surfaci...",yo adanis stock is dropping with more problems...
10278,"bruh, tech IPOs doing well, investors excited 📉",brother new technology ipos are performing wel...
10279,"wtf, Delhi’s AQI at record highs, air quality ...",what the fuck delhis air quality index has hit...


In [37]:
dataset = pd.DataFrame(df)

In [38]:
import pandas as pd
from transformers import pipeline

# Load a sentiment analysis pipeline with a model that supports multiclass sentiment (e.g., 5-class)
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# Function to map sentiment to three categories: Positive, Neutral, Negative
def map_sentiment(score):
    if score in [4, 5]:
        return 'positive'
    elif score == 3:
        return 'neutral'
    else:
        return 'negative'

# Classify each text and map the scores to the three categories
dataset['Polarity'] = dataset['Expanded Meaning'].apply(lambda x: map_sentiment(int(classifier(x)[0]['label'][0])))

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [39]:
# Check the distribution of polarities
print("Polarity Distribution Before Balancing:")
print(dataset['Polarity'].value_counts())

# Polarity Leveling (Balancing)
# You can choose oversampling or undersampling based on the dataset's imbalance
max_samples = dataset['Polarity'].value_counts().max()

# Oversample the minority classes to balance the dataset
balanced_dataset = pd.concat([
    dataset[dataset['Polarity'] == 'positive'],
    dataset[dataset['Polarity'] == 'neutral'].sample(max_samples, replace=True),
    dataset[dataset['Polarity'] == 'negative'].sample(max_samples, replace=True)
])

print("\nPolarity Distribution After Balancing:")
print(balanced_dataset['Polarity'].value_counts())

# Display the balanced dataset
print("\nBalanced Dataset:")
print(balanced_dataset)


Polarity Distribution Before Balancing:
Polarity
negative    5252
positive    3657
neutral     1371
Name: count, dtype: int64

Polarity Distribution After Balancing:
Polarity
neutral     5252
negative    5252
positive    3657
Name: count, dtype: int64

Balanced Dataset:
                                          Informal Text  \
0                        omg, JEE prep is killing me rn   
2                           ttyl, finishing da CAT mock   
4                     sup? u done w/ ur IIT assignment?   
5                       fyi, da UPSC date changed again   
7                     gtg, hv to prep 4 da JEE mock tmr   
...                                                 ...   
6838   no u dont cause u will wake up at 9am for sure .   
4840                                   I lafff the rain   
2452           bruh, da Crab Nebula is cray af up close   
4008  no. my school will start on June1. two days to...   
3199  evryone come here http://www.blogtv.com/People...   

                    

In [40]:
dataset.to_csv('polarity_leveled_dataset.csv')

In [41]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the pretrained BART model and tokenizer
model_name = 'facebook/bart-base'  # You can also use 'bart-large' for better performance
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [43]:
import pandas as pd
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)

# Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(examples['Informal Text'], max_length=128, truncation=True, padding='max_length')
    
    # Tokenize the target (expanded meaning)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['Expanded Meaning'], max_length=128, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10280 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [45]:
# Define training arguments without evaluation strategy
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    evaluation_strategy='no',  # No evaluation during training
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Only training dataset
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.084300
20,0.073700
30,0.078800
40,0.067100
50,0.070400
60,0.068300
70,0.058900
80,0.037700
90,0.039000
100,0.035600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimen

TrainOutput(global_step=3855, training_loss=0.03291883580531592, metrics={'train_runtime': 1183.5149, 'train_samples_per_second': 26.058, 'train_steps_per_second': 3.257, 'total_flos': 2350534046515200.0, 'train_loss': 0.03291883580531592, 'epoch': 3.0})

In [ ]:
# key-wandb.ai: d8b8084404747be261e7c5db146361882b71e7e6

In [54]:
import torch

def generate_expanded_meaning(informal_text):
    # Check if a GPU is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Move the model to the device (GPU or CPU)
    model.to(device)
    
    # Tokenize the input informal text
    inputs = tokenizer([informal_text], return_tensors='pt', max_length=128, truncation=True)
    
    # Move the inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate the output (expanded meaning)
    output_ids = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

    # Decode the generated output
    expanded_meaning = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return expanded_meaning

# Example usage
informal_text = "do it ASAP!"
expanded_meaning = generate_expanded_meaning(informal_text)
print("Expanded Meaning:", expanded_meaning)


Expanded Meaning: do it as soon as possible
